# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since we have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [3]:
df = pd.read_csv("Dataframes/economic_story.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

Instrument        Date  Earnings Per Share - Actual  \
0          AVY.N  2013-01-01                         0.54   
1          AVY.N  2013-04-01                         0.59   
2          AVY.N  2013-07-01                         0.71   
3          AVY.N  2013-10-01                         0.69   
4          AVY.N  2014-01-01                         0.69   
...          ...         ...                          ...   
20115    POOL.OQ  2021-10-01                         4.51   
20116    POOL.OQ  2022-01-01                         2.63   
20117    POOL.OQ  2022-04-01                         4.23   
20118    POOL.OQ  2022-07-01                         7.63   
20119    POOL.OQ  2022-10-01                         4.78   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  \
0                        0.48571                                11.178   
1                        0.57571                                 2.482   
2                        0.70250                                 1.068   
3                        0.63833                                 8.095   
4                        0.68000                                 1.471   
...                          ...                                   ...   
20115                    3.84833                                17.194   
20116                    1.87500                                40.267   
20117                    3.14867                                34.342   
20118                    7.51700                                 1.503   
20119                    4.58750                                 4.196   

      GICS Industry Group Name  Earnings Per Share – Coefficient of Variation  \
0                    Materials                                       3.625620   
1                    Materials                                       2.046169   
2                    Materials                                       1.551601   
3                    Materials                                       2.775994   
4                    Materials                                       3.204412   
...                        ...                                            ...   
20115                Retailing                                       2.836295   
20116                Retailing                                       3.269867   
20117                Retailing                                       6.630736   
20118                Retailing                                       3.625249   
20119                Retailing                                       3.843706   

       Market Capitalization  Revenue - Actual  Enterprise Value  ...  \
0               3.426001e+09      1.532200e+09      4.413001e+09  ...   
1               4.309428e+09      1.498900e+09      5.459128e+09  ...   
2               4.258229e+09      1.552300e+09      5.436229e+09  ...   
3               4.273414e+09      1.504900e+09      5.029514e+09  ...   
4               4.877168e+09      1.583900e+09      5.553068e+09  ...   
...                      ...               ...               ...  ...   
20115           1.742020e+10      1.411448e+09      1.769954e+10  ...   
20116           2.268979e+10      1.035557e+09      2.384882e+10  ...   
20117           1.696712e+10      1.412650e+09      1.843682e+10  ...   
20118           1.405795e+10      2.055818e+09      1.556187e+10  ...   
20119           1.259813e+10      1.615339e+09      1.406160e+10  ...   

       Recommendation - Mean (1-5)  Recommendation - Mean (1-5).1  \
0                          3.00000                          3.000   
1                          2.87500                          3.000   
2                          2.87500                          2.875   
3                          2.62500                          2.875   
4                          2.33333                          2.625   
...                            ...                            ...   
20115                      2.30000                 

In [25]:
#df.dtypes

In [4]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [5]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

Instrument       Date  Earnings Per Share - Actual  \
0          AVY.N 2013-01-01                         0.54   
1          AVY.N 2013-04-01                         0.59   
2          AVY.N 2013-07-01                         0.71   
3          AVY.N 2013-10-01                         0.69   
4          AVY.N 2014-01-01                         0.69   
...          ...        ...                          ...   
20115    POOL.OQ 2021-10-01                         4.51   
20116    POOL.OQ 2022-01-01                         2.63   
20117    POOL.OQ 2022-04-01                         4.23   
20118    POOL.OQ 2022-07-01                         7.63   
20119    POOL.OQ 2022-10-01                         4.78   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  \
0                        0.48571                                11.178   
1                        0.57571                                 2.482   
2                        0.70250                                 1.068   
3                        0.63833                                 8.095   
4                        0.68000                                 1.471   
...                          ...                                   ...   
20115                    3.84833                                17.194   
20116                    1.87500                                40.267   
20117                    3.14867                                34.342   
20118                    7.51700                                 1.503   
20119                    4.58750                                 4.196   

      GICS Industry Group Name  Earnings Per Share – Coefficient of Variation  \
0                    Materials                                       3.625620   
1                    Materials                                       2.046169   
2                    Materials                                       1.551601   
3                    Materials                                       2.775994   
4                    Materials                                       3.204412   
...                        ...                                            ...   
20115                Retailing                                       2.836295   
20116                Retailing                                       3.269867   
20117                Retailing                                       6.630736   
20118                Retailing                                       3.625249   
20119                Retailing                                       3.843706   

       Market Capitalization  Revenue - Actual  Enterprise Value  ...  \
0               3.426001e+09      1.532200e+09      4.413001e+09  ...   
1               4.309428e+09      1.498900e+09      5.459128e+09  ...   
2               4.258229e+09      1.552300e+09      5.436229e+09  ...   
3               4.273414e+09      1.504900e+09      5.029514e+09  ...   
4               4.877168e+09      1.583900e+09      5.553068e+09  ...   
...                      ...               ...               ...  ...   
20115           1.742020e+10      1.411448e+09      1.769954e+10  ...   
20116           2.268979e+10      1.035557e+09      2.384882e+10  ...   
20117           1.696712e+10      1.412650e+09      1.843682e+10  ...   
20118           1.405795e+10      2.055818e+09      1.556187e+10  ...   
20119           1.259813e+10      1.615339e+09      1.406160e+10  ...   

       Recommendation - Mean (1-5)  Recommendation - Mean (1-5).1  \
0                          3.00000                          3.000   
1                          2.87500                          3.000   
2                          2.87500                          2.875   
3                          2.62500                          2.875   
4                          2.33333                          2.625   
...                            ...                            ...   
20115                      2.30000                          2.3

Standardising

In [6]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

Instrument       Date  Earnings Per Share - Actual  \
0          AVY.N 2013-01-01                         0.54   
1          AVY.N 2013-04-01                         0.59   
2          AVY.N 2013-07-01                         0.71   
3          AVY.N 2013-10-01                         0.69   
4          AVY.N 2014-01-01                         0.69   
...          ...        ...                          ...   
20115    POOL.OQ 2021-10-01                         4.51   
20116    POOL.OQ 2022-01-01                         2.63   
20117    POOL.OQ 2022-04-01                         4.23   
20118    POOL.OQ 2022-07-01                         7.63   
20119    POOL.OQ 2022-10-01                         4.78   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  \
0                        0.48571                                11.178   
1                        0.57571                                 2.482   
2                        0.70250                                 1.068   
3                        0.63833                                 8.095   
4                        0.68000                                 1.471   
...                          ...                                   ...   
20115                    3.84833                                17.194   
20116                    1.87500                                40.267   
20117                    3.14867                                34.342   
20118                    7.51700                                 1.503   
20119                    4.58750                                 4.196   

      GICS Industry Group Name  Earnings Per Share – Coefficient of Variation  \
0                    Materials                                       3.625620   
1                    Materials                                       2.046169   
2                    Materials                                       1.551601   
3                    Materials                                       2.775994   
4                    Materials                                       3.204412   
...                        ...                                            ...   
20115                Retailing                                       2.836295   
20116                Retailing                                       3.269867   
20117                Retailing                                       6.630736   
20118                Retailing                                       3.625249   
20119                Retailing                                       3.843706   

       Market Capitalization  Revenue - Actual  Enterprise Value  ...  \
0                   0.001166          0.065123          0.027862  ...   
1                   0.001471          0.064917          0.028207  ...   
2                   0.001453          0.065247          0.028199  ...   
3                   0.001459          0.064955          0.028065  ...   
4                   0.001667          0.065443          0.028238  ...   
...                      ...               ...               ...  ...   
20115               0.006004          0.064377          0.032245  ...   
20116               0.007826          0.062055          0.034274  ...   
20117               0.005848          0.064385          0.032488  ...   
20118               0.004842          0.068358          0.031540  ...   
20119               0.004337          0.065637          0.031045  ...   

       Recommendation - Mean (1-5)  Recommendation - Mean (1-5).1  \
0                          3.00000                          3.000   
1                          2.87500                          3.000   
2                          2.87500                          2.875   
3                          2.62500                          2.875   
4                          2.33333                          2.625   
...                            ...                            ...   
20115                      2.30000                          2.3

In [7]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [8]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

Earnings Per Share - Actual  Earnings Per Share - Mean  \
Instrument Date                                                                 
AVY.N      2013-01-01                         0.54                    0.48571   
           2013-04-01                         0.59                    0.57571   
           2013-07-01                         0.71                    0.70250   
           2013-10-01                         0.69                    0.63833   
           2014-01-01                         0.69                    0.68000   
...                                            ...                        ...   
POOL.OQ    2021-10-01                         4.51                    3.84833   
           2022-01-01                         2.63                    1.87500   
           2022-04-01                         4.23                    3.14867   
           2022-07-01                         7.63                    7.51700   
           2022-10-01                         4.78                    4.58750   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
AVY.N      2013-01-01                                11.178   
           2013-04-01                                 2.482   
           2013-07-01                                 1.068   
           2013-10-01                                 8.095   
           2014-01-01                                 1.471   
...                                                     ...   
POOL.OQ    2021-10-01                                17.194   
           2022-01-01                                40.267   
           2022-04-01                                34.342   
           2022-07-01                                 1.503   
           2022-10-01                                 4.196   

                      GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
AVY.N      2013-01-01                                       3.625620   
           2013-04-01                                       2.046169   
           2013-07-01                                       1.551601   
           2013-10-01                                       2.775994   
           2014-01-01                                       3.204412   
...                                                              ...   
POOL.OQ    2021-10-01                                       2.836295   
           2022-01-01                                       3.269867   
           2022-04-01                                       6.630736   
           2022-07-01                                       3.625249   
           2022-10-01                                       3.843706   

                       Market Capitalization  Revenue - Actual  \
Instrument Date                                                  
AVY.N      2013-01-01               0.001166          0.065123   
           2013-04-01               0.001471          0.064917   
           2013-07-01               0.001453          0.065247   
           2013-10-01               0.001459          0.064955   
           2014-01-01               0.001667          0.065443   
...                                      ...               ...   
POOL.OQ    2021-10-01            

In [10]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_83677/433888285.py:8: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Earnings Per Share – Coefficient of Variation, Market Capitalization, Revenue - Actual, Enterprise Value, Number of Analysts, Recommendation - Mean (1-5), Recommendation - Mean (1-5).1, Price Target - Standard Deviation, Price Close, Enterprise Value To Sales (Daily Time Series Ratio), 3 Month Total Return, Volume, Earnings Per Share - Actual Surprise AbsVals

  fixed_effects_results = fixed_effects_model.fit()


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                   PanelOLS Estimation Summary                                    
==================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                        0.0014
Estimator:                                     PanelOLS   R-squared (Between):              0.0238
No. Observations:                                 17605   R-squared (Within):               0.0019
Date:                                  Wed, Mar 01 2023   R-squared (Overall):              0.0114
Time:                                          19:20:24   Log-likelihood                -7.357e+04
Cov. Estimator:                              Unadjusted                                           
                                                          F-statistic:                      11.946
Entities:                                           502   P-value                           0.0000
Avg Obs:                                         35.070   Distribution:                 F(2,17063)
Min Obs:                                         0.0000                                           
Max Obs:                                         40.000   F-statistic (robust):             11.946
                                                          P-value                           0.0000
Time periods:                                        41   Distribution:                 F(2,17063)
Avg Obs:                                         429.39                                           
Min Obs:                                         36.000                                           
Max Obs:                                         451.00                                           
                                                                                                  
                                  Parameter Estimates                                   
========================================================================================
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
Net Income after Tax  5.011e-10  1.025e-10     4.8879     0.0000   3.001e-10    7.02e-10
Price Target - Mean     -0.0002     0.0011    -0.2124     0.8318     -0.0025      0.0020
========================================================================================

F-test for Poolability: 4.9282
P-value: 0.0000
Distribution: F(539,17063)

Included effects: Entity, Time
"""

In [12]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_83677/699744651.py:8: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Earnings Per Share – Coefficient of Variation, Market Capitalization, Revenue - Actual, Enterprise Value, Number of Analysts, Recommendation - Mean (1-5), Recommendation - Mean (1-5).1, Price Target - Standard Deviation, Price Close, Enterprise Value To Sales (Daily Time Series Ratio), 3 Month Total Return, Volume, Earnings Per Share - Actual Surprise AbsVals

  fixed_effects_results = fixed_effects_model.fit()


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                       PanelOLS Estimation Summary                                        
==========================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise AbsVals   R-squared:                        0.0017
Estimator:                                             PanelOLS   R-squared (Between):             -0.0682
No. Observations:                                         17605   R-squared (Within):              -0.0028
Date:                                          Wed, Mar 01 2023   R-squared (Overall):             -0.0451
Time:                                                  19:20:50   Log-likelihood                -6.886e+04
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      14.381
Entities:                                                   502   P-value                           0.0000
Avg Obs:                                                 35.070   Distribution:                 F(2,17063)
Min Obs:                                                 0.0000                                           
Max Obs:                                                 40.000   F-statistic (robust):             14.381
                                                                  P-value                           0.0000
Time periods:                                                41   Distribution:                 F(2,17063)
Avg Obs:                                                 429.39                                           
Min Obs:                                                 36.000                                           
Max Obs:                                                 451.00                                           
                                                                                                          
                                  Parameter Estimates                                   
========================================================================================
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
Net Income after Tax -1.984e-10  7.845e-11    -2.5291     0.0114  -3.522e-10  -4.464e-11
Price Target - Mean     -0.0041     0.0009    -4.6190     0.0000     -0.0058     -0.0023
========================================================================================

F-test for Poolability: 13.379
P-value: 0.0000
Distribution: F(539,17063)

Included effects: Entity, Time
"""